In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

# Load dataset (CSV must have columns 'Year' and 'Level')
df = pd.read_csv('data/groundwater_levels.csv')
df['Year'] = pd.to_datetime(df['Year'], format='%Y')
df.set_index('Year', inplace=True)

# Split data: 80% train, 20% test
split_idx = int(len(df)*0.8)
train, test = df.iloc[:split_idx], df.iloc[split_idx:]

# ========== 1. Fit ARIMA on train data ==========
# Choose ARIMA order (p,d,q) - you can tune this or use auto_arima
arima_order = (2,1,2)
arima_model = ARIMA(train['Level'], order=arima_order)
arima_result = arima_model.fit()

# ARIMA forecast on train + test period (to get residuals)
arima_train_pred = arima_result.predict(start=train.index[0], end=train.index[-1], typ='levels')
arima_test_pred = arima_result.predict(start=test.index[0], end=test.index[-1], typ='levels')

# ========== 2. Train ANN on residuals ==========
# Residuals (errors ARIMA didn't explain)
residuals = train['Level'] - arima_train_pred

# Prepare features for ANN - here, we'll use lagged residuals as inputs
def create_lagged_features(series, lag=3):
    X, y = [], []
    for i in range(lag, len(series)):
        X.append(series[i-lag:i])
        y.append(series[i])
    return np.array(X), np.array(y)

lag = 3
X_train, y_train = create_lagged_features(residuals.values, lag=lag)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# ANN model
ann = MLPRegressor(hidden_layer_sizes=(50,30), max_iter=1000, random_state=42)
ann.fit(X_train_scaled, y_train)

# Predict residuals on test data
# Prepare test features similarly
# First, get residuals for test period (we won't have actual residuals, so use previous predictions + lagged residuals)
combined_residuals = np.concatenate([residuals.values, np.zeros(len(test))])

X_test = []
for i in range(len(residuals), len(residuals)+len(test)):
    if i - lag < 0:
        # Pad with zeros if insufficient history (usually not needed here)
        X_test.append(np.zeros(lag))
    else:
        X_test.append(combined_residuals[i-lag:i])
X_test = np.array(X_test)
X_test_scaled = scaler.transform(X_test)

ann_residuals_pred = ann.predict(X_test_scaled)

# ========== 3. Combine ARIMA prediction and ANN residual prediction ==========
final_pred = arima_test_pred.values + ann_residuals_pred

# ========== 4. Evaluation ==========
mse = mean_squared_error(test['Level'], final_pred)
r2 = r2_score(test['Level'], final_pred)
print(f'Mean Squared Error: {mse:.4f}')
print(f'R^2 Score: {r2:.4f}')

# ========== 5. Plot ==========
plt.figure(figsize=(12,6))
plt.plot(train.index, train['Level'], label='Train Actual')
plt.plot(test.index, test['Level'], label='Test Actual')
plt.plot(test.index, arima_test_pred, label='ARIMA Prediction')
plt.plot(test.index, final_pred, label='Combined ARIMA + ANN Prediction', linestyle='--')
plt.xlabel('Year')
plt.ylabel('Groundwater Level')
plt.title('Groundwater Level Prediction: ARIMA + ANN')
plt.legend()
plt.show()
